In [152]:
import numpy as np
import random
from IPython.display import Image, display 
from graphviz import Digraph

# Define constants, variables, and operators
constant_range = np.linspace(-10, 10, num=50)
CONSTANTS = list(constant_range)
print(CONSTANTS)

def safe_divide(a, b):
    return np.divide(a, b, where=b != 0, out=np.full_like(a, np.nan))

def safe_sqrt(a):
    return np.sqrt(a, where=a >= 0, out=np.full_like(a, np.nan)) 

def safe_log(a):
    return np.log(a, where=a > 0, out=np.full_like(a, -np.inf)) 

UNARY_OPERATORS = {
    'sin': np.sin,
    'cos': np.cos,
    'tan': np.tan,
    'sqrt': safe_sqrt,
    'log': safe_log,
    'exp': np.exp,    
    'square': np.square,
}   

BINARY_OPERATORS = {
    '+': np.add,
    '-': np.subtract,
    '*': np.multiply,
    '/': safe_divide,
    "^": np.power
}

[np.float64(-10.0), np.float64(-9.591836734693878), np.float64(-9.183673469387756), np.float64(-8.775510204081632), np.float64(-8.36734693877551), np.float64(-7.959183673469388), np.float64(-7.551020408163265), np.float64(-7.142857142857142), np.float64(-6.73469387755102), np.float64(-6.326530612244898), np.float64(-5.918367346938775), np.float64(-5.5102040816326525), np.float64(-5.1020408163265305), np.float64(-4.6938775510204085), np.float64(-4.285714285714286), np.float64(-3.8775510204081627), np.float64(-3.4693877551020407), np.float64(-3.0612244897959187), np.float64(-2.6530612244897958), np.float64(-2.244897959183673), np.float64(-1.8367346938775508), np.float64(-1.4285714285714288), np.float64(-1.020408163265305), np.float64(-0.612244897959183), np.float64(-0.204081632653061), np.float64(0.204081632653061), np.float64(0.612244897959183), np.float64(1.0204081632653068), np.float64(1.4285714285714288), np.float64(1.8367346938775508), np.float64(2.2448979591836746), np.float64(2.65

Node class

In [153]:
class Node:
    def __init__(self, value, is_operator=False):
        """
        Initialize a Node.
        
        :param value: The value of the node (operator, constant, or variable name).
        :param is_operator: True if the node represents an operator, otherwise False.
        """
        self.value = value
        self.is_operator = is_operator
        self.left = None
        self.right = None

    def evaluate(self, variables):
        """
        Evaluate the subtree rooted at this node.
        
        :param variables: A dictionary mapping variable names to their values.
        :return: The result of evaluating the subtree.
        """
        # Check for None values to avoid errors
        if self is None:
            return 0

        if self.is_operator:
            # Unary operator (e.g., sin, log, etc.)
            if self.value in list(UNARY_OPERATORS.keys()):    
                if self.left is None:
                    raise ValueError(f"Missing left child for operator {self.value}")
                left_value = self.left.evaluate(variables)
                return UNARY_OPERATORS[self.value](left_value)
            
            # Binary operator (e.g., add, sub, mul, div, etc.)
            elif self.value in list(BINARY_OPERATORS.keys()):
                if self.left is None or self.right is None:
                    raise ValueError(f"Missing children for operator {self.value}")
                left_value = self.left.evaluate(variables)
                right_value = self.right.evaluate(variables)
                return BINARY_OPERATORS[self.value](left_value, right_value)
        
        elif self.value in variables:
            # Variable node (e.g., x0, x1, etc.)
            return variables[self.value]
        
        # Constant node (just return the constant value)
        return self.value

Individual class

In [154]:
class Individual:
    def __init__(self, root, file_path):
        """
        Initialize an Individual (a tree).
        
        :param root: The root node of the tree.
        """
        self.root = root

    def evaluate(self, variables):
        """
        Evaluate the tree.
        
        :param variables: A dictionary mapping variable names to their values.
        :return: The result of evaluating the tree.
        """
        return self.root.evaluate(variables)
    
    def tree_size(self):
        """
        Return the number of nodes in the tree.
        """
        def count_nodes(node):
            if node is None:
                return 0
            return 1 + count_nodes(node.left) + count_nodes(node.right)
        
        return count_nodes(self.root)

    def mse(self, file_path):
        """
        Calculate the mse of the individual using Mean Squared Error (MSE).
        
        :param file_path: Path to the .npz file containing 'x' and 'y'.
        :return: The MSE between the predicted and actual y values.
        """
        # Load the data
        data = np.load(file_path)
        x = data['x']
        y = data['y']

        # Initialize variables for prediction
        num_features = x.shape[0]
        variables = {f'x{i}': None for i in range(num_features)}

        # Compute predictions
        y_pred = []
        for i in range(x.shape[1]):  # Iterate over each column
            for j in range(num_features):  # Set variable values for this row
                variables[f'x{j}'] = x[j, i]
            y_pred.append(self.evaluate(variables))
        y_pred = np.array(y_pred)

        # Calculate MSE
        mse = np.mean((y - y_pred) ** 2)
        self.mse_value = mse
        return mse
    
    # Variant of mse involving complexity penalty
    def mse_with_complexity(self, file_path, complexity_penalty=0.05):
        """
        Calculate the fitness with an adaptive complexity penalty.

        :param file_path: Path to the .npz file containing 'x' and 'y'.
        :param complexity_penalty: The penalty factor for complexity.
        :return: The fitness with the complexity penalty included.
        """
        base_mse = self.mse(file_path)
        
        if self.tree_size() == 1:
            return base_mse*2  # Penalize trees with only one node
        
        complexity = complexity_penalty * self.tree_size()
        
        mse_with_complexity = base_mse * (1 + complexity)
        return mse_with_complexity

    def get_all_nodes(self):
        """
        Retrieve all nodes in the tree.

        :return: A list of all nodes in the tree.
        """
        def collect_nodes(node):
            if node is None:
                return []
            return [node] + collect_nodes(node.left) + collect_nodes(node.right)
        
        return collect_nodes(self.root)
    

    def __str__(self):
        """
        Return a string representation of the tree.
        """
        return self._str_helper(self.root)

    def _str_helper(self, node):
        """
        Helper function for string representation. Recursively traverse the tree.
        """
        if node is None:
            return ""
        
        if node.is_operator:
            if node.right is not None:
                return f"({self._str_helper(node.left)} {node.value} {self._str_helper(node.right)})"
            else:
                return f"{node.value}({self._str_helper(node.left)})"
        return str(node.value)

In [155]:
def mse_percentage(mse_value, file_path):
        """
        Calculate the MSE percentage to assess how logical the MSE value is.

        :param file_path: Path to the .npz file containing 'x' and 'y'.
        :return: The MSE percentage.
        """
        
        # Load the data to get the y values for min and max
        data = np.load(file_path)
        y = data['y']
        
        # Calculate the range of y values
        y_max = np.max(y)
        y_min = np.min(y)
        
        # Calculate the MSE percentage
        mse_percentage = (mse_value / (y_max - y_min)) * 100
        
        return mse_percentage

Create tree

In [156]:
# Create a random tree out of the class Individual
def create_tree(num_nodes, file_name):
    """
    Create a random tree with the specified number of nodes.
    
    :param num_nodes: The number of nodes in the tree.
    :param file_name: The .npz file containing the data.
    :return: A random tree as an Individual object.
    """
    # Assuming x_data is loaded from 'problem_20.npz', let's load it properly
    data = np.load(file_name)
    x_data = data['x']  # Assuming 'x' is the key for your data in the .npz file
    y_data = data['y']  # Assuming 'y' is the key for your labels (if you need it)

    # Make sure x_data is a NumPy array
    x_data = np.array(x_data)

    num_vars = x_data.shape[0]  # Number of variables (rows in x)
    VARIABLES = [f'x{i}' for i in range(num_vars)]  # Generate the variable names dynamically
    variable_included = False  # Flag to check if at least one variable is included in the tree

    def random_node(is_operator=False):
        """Helper function to generate a random node (operator or constant/variable)."""
        nonlocal variable_included
        if is_operator:
            # Random operator node (should never be a terminal node)
            value = random.choice(list(UNARY_OPERATORS.keys())+list(BINARY_OPERATORS.keys()))
            return Node(value, is_operator=True)
        else:
            # Random constant or variable node (only for leaf nodes)
            value = random.choice(CONSTANTS) if random.random() < 0.3 else random.choice(VARIABLES)
            if value in VARIABLES:
                variable_included = True
            return Node(value, is_operator=False)

    def build_tree(nodes_left, is_root=True):
        """Recursively build a tree. Ensure the first node is not a terminal node."""
        if nodes_left == 1:
            # If only one node is left, it should be a terminal node (constant or variable)
            return random_node(is_operator=False), 1
        else:
            # Choose an operator (either unary or binary)
            operator = random.choice(list(UNARY_OPERATORS.keys()) + list(BINARY_OPERATORS.keys()))
            node = Node(operator, is_operator=True)

            if operator in list(UNARY_OPERATORS.keys()):
                # Unary operator: Only one child is required
                right_subtree_size = nodes_left - 1
                right_child, _ = build_tree(right_subtree_size, is_root=False)
                node.left = right_child
                return node, 1 + right_subtree_size  # 1 for the operator, + subtree size
            elif operator in list(BINARY_OPERATORS.keys()):
                # Binary operator: Two children are required
                if nodes_left <= 2:
                    # If there are only two nodes left, force it to be a terminal node
                    left_subtree_size = 1
                    right_subtree_size = 1
                else:
                    right_subtree_size = random.randint(1, nodes_left - 2)
                    left_subtree_size = nodes_left - 1 - right_subtree_size

                # Build left and right subtrees
                left_child, left_size = build_tree(left_subtree_size, is_root=False)
                right_child, right_size = build_tree(right_subtree_size, is_root=False)

                node.left = left_child
                node.right = right_child

                return node, 1 + left_size + right_size  # 1 for the operator, + left and right subtree sizes

    # Build the tree and return it as an Individual object
    root_node, _ = build_tree(num_nodes)
    
    # Ensure at least one variable is included
    if not variable_included:
        print("No variable included in the tree; retrying...")
        return create_tree(num_nodes, file_name)  # Retry tree creation if no variable was included
    
    return Individual(root_node, file_name)

In [157]:
def draw_tree(tree):
    dot = Digraph(comment='Tree')
    
    # Set to track unique nodes by their value, but we're not excluding repeats
    node_counter = {}  # This will help track how many times a node has been used
    
    def add_edges(node, parent=None):
        if node:
            # Create a unique identifier for the node based on its value and repetition count
            node_name = str(node.value)
            
            # Track how many times we've seen this node (for repeating nodes)
            if node_name not in node_counter:
                node_counter[node_name] = 0
            node_counter[node_name] += 1

            # Append the repetition count to make the node name unique
            unique_node_name = f"{node_name}_{node_counter[node_name]}"

            # Add node to the graph if it has not been added
            dot.node(unique_node_name, label=str(node.value))
            
            if parent:
                # Connect parent to this node
                dot.edge(str(parent), unique_node_name)

            # Recursively add left and right children
            if node.left:
                add_edges(node.left, unique_node_name)
            if node.right:
                add_edges(node.right, unique_node_name)

    # Start from the root of the tree
    add_edges(tree.root)

    # Render the graph to a file and view it
    #dot.render('tree_visualization')
    
    img_data = dot.pipe(format='png')
    display(Image(img_data))


Population class

In [158]:
class Population:
    def __init__(self, size, num_nodes, file_name, individuals=None):
        """
        Initialize a population of random individuals (trees).
        
        :param size: Number of individuals in the population.
        :param num_nodes: Number of nodes for each tree.
        :param file_name: the .npz file containing the data
        """
        # self.individuals = [
        #     create_tree(num_nodes, x_data) for _ in range(size)
        # ]
        
        # Assuming x_data is loaded from 'problem_20.npz', let's load it properly
        data = np.load(file_name)
        x_data = data['x']  # Assuming 'x' is the key for your data in the .npz file
        y_data = data['y']  # Assuming 'y' is the key for your labels (if you need it)

        # Make sure x_data is a NumPy array
        x_data = np.array(x_data)
        
        self.size = size
        
        if individuals:
            self.individuals = individuals
        else:
            self.individuals = []
            unique_individuals = set()
            
            while len(self.individuals) < size:
                individual = create_tree(num_nodes, file_name)
                mse_value = individual.mse(file_name)
                
                # Check individual is valid
                if np.isnan(mse_value) or np.isinf(mse_value):
                    continue
                
                tree_representation = str(individual)
                
                if tree_representation not in unique_individuals and individual.tree_size() == num_nodes:
                    self.individuals.append(individual) 
                    unique_individuals.add(tree_representation)
                

    # def evaluate_mse(self, file_path, complexity_penalty=0.01):
    #     """
    #     Evaluate the mse of all individuals in the population.
        
    #     :param file_path: Path to the .npz file containing 'x' and 'y'.
    #     :param complexity_penalty: Penalty for tree complexity.
    #     """
    #     for individual in self.individuals:
    #         individual.mse_value = individual.mse(file_path)
        
    def get_best_individual(self, file_name):
        """
        Get the individual with the best mse in the population.
        
        :return: The best individual.
        """
        valid_individuals = [ind for ind in self.individuals if not np.isnan(ind.mse(file_name)) and not np.isinf(ind.mse(file_name))]
        if valid_individuals:
            best_individual = min(valid_individuals, key=lambda ind: ind.mse(file_name))
        else:
            print("No valid individuals found in the population.")
        return best_individual

    def get_worst_individual(self, file_name):
        """
        Get the individual with the best mse in the population.
        
        :return: The best individual.
        """
        return max(self.individuals, key=lambda ind: ind.mse(file_name))

    def get_average_mse(self):
        """
        Get the average mse of the population.
        
        :return: Average mse value.
        """
        return np.mean([ind.mse_value for ind in self.individuals])
    
    # def update_pareto_front(self):
    #     """
    #     Update the pareto front of the population.
    #     """
    #     pareto_front = []
    #     for individual in self.individuals:
    #         dominated = False
    #         for other in pareto_front:
    #             if other.mse_value < individual.mse_value and other.tree_size() <= individual.tree_size():
    #                 dominated = True
    #                 break
    #         if not dominated:
    #             pareto_front.append(individual)
    #         self.pareto_front = pareto_front
    
    def tournament_selection(self, tournament_size=3):
        """
        Perform probabilistic tournament selection to choose an individual from the population.
        
        :param tournament_size: Number of individuals in the tournament.
        :return: The selected individual.
        """
        # Randomly select individuals for the tournament
        tournament = random.sample(self.individuals, tournament_size)
        
        # Individual with best mse
        winner = min(tournament, key=lambda ind: ind.mse_value)

        return winner
    
    def tournament_selection2(self, tournament_size=3, selection_probability=0.8):
        """
        Perform probabilistic tournament selection to choose an individual from the population.
        probability will roughly be p, p(1-p), p(1−p)^2,...for the first, second, third fittest, and so on
        
        :param tournament_size: Number of individuals in the tournament.
        :param selection_probability: Probability of selecting the fittest individual in the tournament.
        :return: The selected individual.
        """
        # Randomly select individuals for the tournament
        tournament = random.sample(self.individuals, tournament_size)
        
        # Sort the tournament individuals by mse (ascending order for minimization problems)
        sorted_tournament = sorted(tournament, key=lambda ind: ind.mse_value)       
        
        while len(sorted_tournament) > 1:
            # Select the fittest individual (first in sorted list)
            fittest_individual = sorted_tournament[0]
            
            # Decide whether to select this individual
            if random.random() < selection_probability:
                return fittest_individual
            
            # If not selected, remove the fittest individual and continue
            sorted_tournament.pop(0)
    
        # If only one individual is left, return it
        return sorted_tournament[0]

In [159]:
def get_random_node(tree):
    """
    Select a random node from the tree.
    
    :return: A randomly selected node from the tree.
    """
    all_nodes = tree.get_all_nodes()
    return random.choice(all_nodes) if all_nodes else None

def rebuild_subtree(node):
    """
    Rebuild a subtree starting from the given node, replicating it exactly.
    
    :param node: The root node of the subtree to replicate.
    :return: A new Node that is a copy of the subtree rooted at `node`.
    """
    if node is None:
        return None

    # Create a copy of the current node
    new_node = Node(value=node.value, is_operator=node.is_operator)

    # Recursively copy the left and right children
    new_node.left = rebuild_subtree(node.left)
    new_node.right = rebuild_subtree(node.right)

    return new_node

def extract_and_rebuild_subtree(tree):
    """
    Extract and rebuild a random subtree from the tree.

    :return: A new subtree that is a copy of the randomly chosen subtree.
    """
    random_node = get_random_node(tree)
    if random_node is None:
        raise ValueError("Tree is empty; cannot extract a subtree.")

    # Rebuild the subtree starting from the selected node
    subtree = Individual(rebuild_subtree(tree, random_node))
    return subtree


def replace_subtree(tree, target_node, new_subtree):
    """
    Replace a subtree in the tree with a new subtree.
    
    :param tree: The tree (Individual) to modify.
    :param target_node: The node where the replacement should happen.
    :param new_subtree: The new subtree to replace the old one.
    :return: The modified tree (Individual).
    """
    # Find the parent node of the target node
    parent_node = None
    stack = [tree.root]
    while stack:
        current_node = stack.pop()
        if current_node.left == target_node or current_node.right == target_node:
            parent_node = current_node
            break
        if current_node.left:
            stack.append(current_node.left)
        if current_node.right:
            stack.append(current_node.right)

    # Replace the target node with the new subtree
    if parent_node:
        if parent_node.left == target_node:
            parent_node.left = new_subtree
        else:
            parent_node.right = new_subtree
    else:
        # If the target node is the root, replace the entire tree
        tree.root = new_subtree
    return tree


def perform_crossover(tree1, tree2):
    """
    Perform subtree crossover between two trees.
    
    :param tree1: The first tree (Individual) to crossover.
    :param tree2: The second tree (Individual) to crossover.
    :param x_data: The x_data to use for generating new subtrees.
    :return: Two new trees resulting from the crossover operation.
    """        
    node1 = get_random_node(tree1)
    node2 = get_random_node(tree2)
    
    if node1 is None or node2 is None:
        raise ValueError("One or both trees are empty; cannot perform crossover.")
    
    # Extract subtrees from each tree
    subtree1 = rebuild_subtree(node1)
    subtree2 = rebuild_subtree(node2)
    
    # Replace subtrees in the opposite trees
    new_tree1 = replace_subtree(tree1, node1, subtree2)
    new_tree2 = replace_subtree(tree2, node2, subtree1)
    
    return new_tree1, new_tree2

In [160]:
def new_subtree_mutation(tree, file_name):
    """
    Perform subtree mutation on a given tree.
    
    :param tree: The tree to mutate.
    :param file_name: The .npz file containing the data.
    """
    # Get a random node from the tree
    target_node = get_random_node(tree)
    
    if target_node is None:
        raise ValueError("Tree is empty; cannot perform mutation.")

    # Calculate the size of the subtree rooted at the selected node
    subtree_individual = Individual(target_node, file_name)
    subtree_size = subtree_individual.tree_size()

    # Create a new subtree with the same size
    new_subtree = create_tree(subtree_size, file_name).root

    # Replace the selected subtree with the new one
    replace_subtree(tree, target_node, new_subtree)
    
    return tree

def constant_mutation(tree, file_name):
    """"
    Perform constant mutation on a given tree by multiplying a constant by another one.
    
    :param tree: The tree to mutate.
    """
    # Get a random node from the tree
    target_node = get_random_node(tree)
    
    if target_node is None:
        raise ValueError("Tree is empty; cannot perform mutation.")
    
    # If the target node is a constant node, replace it with a new constant
    if target_node.value in CONSTANTS:
        target_node.value *= random.choice(CONSTANTS)
    
    return tree

def operator_mutation(tree, file_name):
    """"
    Perform operator mutation on a given tree by replacing an operator with another one of the same degree.
    
    :param tree: The tree to mutate.
    """
    # Get a random node from the tree
    target_node = get_random_node(tree)
    if target_node is None:
        raise ValueError("Tree is empty; cannot perform mutation.")
    
    while target_node.is_operator == False:
        target_node = get_random_node(tree)
        
    # If the target node is an operator node, replace it with a new operator of the same degree
    if target_node.is_operator:
        new_value = target_node.value
        if target_node.value in list(UNARY_OPERATORS.keys()):
            while new_value == target_node.value:
                new_value = random.choice(list(UNARY_OPERATORS.keys())) 
        elif target_node.value in list(BINARY_OPERATORS.keys()):
            while new_value == target_node.value:
                new_value = random.choice(list(BINARY_OPERATORS.keys()))
                
        target_node.value = new_value
    
    return tree


def append_mutation(tree, file_name):
    """
    Append a node to a leaf of the tree.
    
    :param tree: The root node of the tree.
    :param file_name: The file name for generating new nodes.
    """
    # Generate a random leaf node
    data = np.load(file_name)
    x_data = data['x']
    variables = [f'x{i}' for i in range(x_data.shape[0])]
    
    leaf_nodes = [node for node in tree.get_all_nodes() if node.left is None and node.right is None]
    if leaf_nodes:
        random_leaf = random.choice(leaf_nodes)
        
    if random.random() < 0.5:
        # Binary operator case
        new_operator = Node(random.choice(list(BINARY_OPERATORS.keys())), is_operator=True)
        new_node = Node(random.choice(CONSTANTS + variables), is_operator=False)
        
        if random.random() < 0.5:
            new_operator.left = random_leaf
            new_operator.right = new_node
            new_tree = Individual(new_operator, file_name) 
            mutated_tree = replace_subtree(tree, random_leaf, new_tree.root)
        else:
            new_operator.left = new_node
            new_operator.right = random_leaf
            new_tree = Individual(new_operator, file_name)
            mutated_tree = replace_subtree(tree, random_leaf, new_tree.root)
    else:
        new_operator = Node(random.choice(list(UNARY_OPERATORS.keys())), is_operator=True)
        new_operator.left = random_leaf
        new_operator.right = None
        new_tree = Individual(new_operator, file_name)
        mutated_tree = replace_subtree(tree, random_leaf, new_tree.root)

    return mutated_tree

def prepend_mutation(tree, file_name):
    """
    Prepend a node to the root of the tree.
    
    :param tree: The root node of the tree.
    :param file_name: The file name for generating new nodes.
    """
    if random.random() < 0.5:
        # Binary operator case
        new_root = Node(random.choice(list(BINARY_OPERATORS.keys())), is_operator=True)
        
        # Generate a random leaf node
        data = np.load(file_name)
        x_data = data['x']
        variables = [f'x{i}' for i in range(x_data.shape[0])]

        new_node = Node(random.choice(CONSTANTS + variables), is_operator=False)
        
        if random.random() < 0.5:
            new_root.left = Node('x0', is_operator=False)
            new_root.right = new_node
            new_tree = Individual(new_root, file_name) 
            mutated_tree = replace_subtree(new_tree, new_root.left, tree.root)
        else:
            new_root.left = new_node
            new_root.right = Node('x0', is_operator=False)
            new_tree = Individual(new_root, file_name)
            mutated_tree = replace_subtree(new_tree, new_root.right, tree.root)
    else:
        new_root = Node(random.choice(list(UNARY_OPERATORS.keys())), is_operator=True)
        new_root.left = Node('x0', is_operator=False)
        new_root.right = None
        new_tree = Individual(new_root, file_name)
        mutated_tree = replace_subtree(new_tree, new_root.left, tree.root)

    return mutated_tree

def delete_subtree_mutation(tree, file_name):
    """
    Delete a random subtree from the tree.
    
    :param tree: The root node of the tree.
    """
    data = np.load(file_name)
    x_data = data['x']
    variables = [f'x{i}' for i in range(x_data.shape[0])]
    # Get a random node from the tree
    target_node = get_random_node(tree)
    new_node = Node(random.choice(CONSTANTS + variables), is_operator=False)
    
    if target_node is None:
        raise ValueError("Tree is empty; cannot perform mutation.")
    
    # Replace the selected subtree with None
    replace_subtree(tree, target_node, new_node)
    
    return tree


In [161]:
def perform_mutation(tree, file_name):
    """
    Perform a mutation operation on the tree based on the given probability.
    
    :param tree: The tree to mutate.
    :param mutation_probability: The probability of performing a mutation.
    :param file_name: The file name for generating new nodes.
    """
    mutation_type = random.choice(['subtree', 'constant', 'operator', 'append', 'prepend', 'delete'])
    if mutation_type == 'subtree':
        return new_subtree_mutation(tree, file_name)
    elif mutation_type == 'constant':
        return constant_mutation(tree, file_name)
    elif mutation_type == 'operator':
        return operator_mutation(tree, file_name)
    elif mutation_type == 'append':
        return append_mutation(tree, file_name)
    elif mutation_type == 'prepend':
        return prepend_mutation(tree, file_name)
    elif mutation_type == 'delete':
        return delete_subtree_mutation(tree, file_name)
    return tree

SYMBOLIC REGRESSION FUNCTION

In [162]:
def symbolic_regression(
    file_name,
    population_size=100,
    num_generations=2,
    num_nodes=10,
    elite_size=5,
    tournament_size=3,
    crossover_probability=0.2,
    complexity_penalty=0.01
):
    """
    Evolve a population of trees using genetic programming.

    :param file_name: Path to the .npz file containing 'x' and 'y' data.
    :param population_size: Number of individuals in the population.
    :param num_generations: Number of generations to symbolic_regression.
    :param num_nodes: Number of nodes for each tree.
    :param tournament_size: Size of the tournament for selection.
    :param mutation_probability: Probability of applying mutation.
    :param crossover_probability: Probability of applying crossover.
    :param complexity_penalty: Penalty for tree complexity during fitness evaluation.
    :return: The best individual after evolution.
    """
    # Step 1: Initialize population
    population = Population(size=population_size, num_nodes=num_nodes, file_name=file_name)
    
    for generation in range(num_generations):
        print(f"\n--- Generation {generation + 1} ---")
        
        sorted_population = sorted(population.individuals, key=lambda ind: ind.mse_with_complexity(file_name))
        for individual in sorted_population:
            print(individual, individual.mse_with_complexity(file_name))
        
        # Create the next generation 
        new_population = []
        for i in range(elite_size):
            elite = sorted_population[i]
            copy_elite = Individual(rebuild_subtree(elite.root), file_name)
            new_population.append(copy_elite)

        temp_population = set()
        while len(temp_population) < population_size - elite_size:
            parent1 = population.tournament_selection2(tournament_size)

            # Crossover and Mutation
            if random.random() < crossover_probability:
                parent2 = population.tournament_selection2(tournament_size)
                offspring1, offspring2 = perform_crossover(parent1, parent2)
                if not np.isnan(offspring1.mse_with_complexity(file_name)) and not np.isinf(offspring1.mse_with_complexity(file_name)):
                    temp_population.add(offspring1)

                if not np.isnan(offspring2.mse_with_complexity(file_name)) and not np.isinf(offspring2.mse_with_complexity(file_name)):
                    temp_population.add(offspring2)
            else:
                offspring = perform_mutation(parent1, file_name)
                if not np.isnan(offspring.mse_with_complexity(file_name)) and not np.isinf(offspring.mse_with_complexity(file_name)):
                    temp_population.add(offspring)
        
        sorted_temp_population = sorted(list(temp_population), key=lambda ind: ind.mse_with_complexity(file_name))
        new_population.extend(sorted_temp_population[:population_size - elite_size])
        
        # Filter individuals of complexity 1
        new_population = [ind for ind in new_population if not ind.tree_size() == 1]
        
        # Replace the old population with the new one
        population.individuals = new_population[:population_size]

    # Return the best individual from the final population
    return population.get_best_individual(file_name)


RUN SYMBOLIC REGRESSION

In [163]:
best_individual = symbolic_regression(
    file_name='../data/problem_1.npz',
    population_size=100,
    num_generations=30,
    num_nodes=10,
    elite_size=5,
    tournament_size=5,
    crossover_probability=0.5,
    complexity_penalty=0.01
)

print("Best Individual:", best_individual)

AttributeError: 'Individual' object has no attribute 'evaluate'

In [133]:
# def create_generation(current_population, file_name, crossover_rate=0.8, elite_size = 3):
#     """
#     Create a new generation of individuals using mutation and crossover.

#     :param current_population: The current population of individuals.
#     :param file_name: The file name for mse calculations.
#     :param crossover_rate: The probability of performing crossover.
#     :param elite_size: The number of top individuals to retain.
#     :return: A new Population object representing the next generation.
#     """
#     # Sort population by mse and copy elites
#     sorted_population = sorted(current_population.individuals, key=lambda ind: ind.mse(file_name))
#     new_generation = []
    
#     for i in range(elite_size):
#         elite = sorted_population[i]
#         elite_copy = Individual(rebuild_subtree(elite.root), file_name)
#         new_generation.append(elite_copy)
        
#     # Create the rest of the new generation
#     while len(new_generation) < len(current_population.individuals):        
#         # Perform crossover with probability `crossover_rate`
#         if random.random() < crossover_rate:
#             # Perform tournament selection to choose parents
#             parent1 = current_population.tournament_selection()
#             parent2 = current_population.tournament_selection()
#             while (parent1 == parent2):
#                 parent2 = current_population.tournament_selection()
            
#             try:    
#                 child1, child2 = perform_crossover(parent1, parent2)
#                 if not np.isnan(child1.mse(file_name)) and not np.isinf(child1.mse(file_name)):
#                     child1_copy = Individual(rebuild_subtree(child1, child1.root), file_name)
#                     if len(new_generation) < len(current_population.individuals):
#                         new_generation.append(child1_copy)
#                 if not np.isnan(child2.mse(file_name)) and not np.isinf(child2.mse(file_name)):
#                     child2_copy = Individual(rebuild_subtree(child2, child2.root), file_name)
#                     if len(new_generation) < len(current_population.individuals):
#                         new_generation.append(child2_copy)
#             except:
#                 continue        
#         else:
#             # Perform mutation
#             selected_individual = current_population.tournament_selection()
#             try:
#                 mutated = perform_mutation(selected_individual, file_name)
#                 if not np.isnan(mutated.mse(file_name)) and not np.isinf(mutated.mse(file_name)):
#                     mutated_copy = Individual(rebuild_subtree(mutated, mutated.root), file_name)
#                     if len(new_generation) < len(current_population.individuals):
#                         new_generation.append(mutated_copy)                       
#             except:
#                 continue
    
#     new_generation = new_generation[:len(current_population.individuals)]
#     new_population = Population(size=0, num_nodes=tree_size, file_name=file_name, individuals=new_generation)

#     # Verify that the best mse is preserved
#     new_best = min(ind.mse(file_name) for ind in new_population.individuals)
#     old_best = min(ind.mse(file_name) for ind in current_population.individuals)
#     assert new_best <= old_best, f"Best mse got worse from {old_best} to {new_best}"
    
#     return new_population

In [134]:
# #parameters
# file_name = '../data/problem_1.npz'
# population_size = 100
# elite_size = 3
# tree_size = 10
# num_generations = 100

# current_population = Population(size=population_size, num_nodes=tree_size, file_name=file_name)

# best_individual = current_population.get_best_individual()
# print("Initial Best Individual:", best_individual)
# print("Initial Best MSE Value:", best_individual.mse(file_name))
# print("Initial MSE percentage", mse_percentage(best_individual.mse(file_name), file_name))

# for generation in range(num_generations):
#     new_population = create_generation(current_population, file_name, elite_size=elite_size)
            
#     # for ind in new_population.individuals:
#     #     print(str(ind) + ":" + str(ind.mse_value))
    
#     new_best_individual = new_population.get_best_individual()
#     print(f"Generation {generation+1}")
#     print("Best Individual:", new_best_individual)
#     print("Best MSE Value:", new_best_individual.mse(file_name))
#     current_population.individuals = new_population.individuals[:]
            
# print("MSE percentage", mse_percentage(new_population.get_best_individual().mse_value, file_name))

TEST RESULTS

In [ ]:
n1 = Node('sin', is_operator=True)
n1.left = Node('x0', is_operator=False)  # x₀

formula = Individual(n1, '../data/problem_1.npz')

print(formula)
mse_value = formula.mse('../data/problem_1.npz')
print("MSE value", mse_value)
mse_value_percentage = mse_percentage(mse_value, '../data/problem_1.npz')
print("MSE percentage:", mse_value_percentage)

In [ ]:
# Create the nodes for the expression
n2 = Node('*', is_operator=True)  # Outer multiplication
n2.right = Node('x0', is_operator=False)  # x₀ as the right operand

# Left side of the outermost multiplication
n2.left = Node('+', is_operator=True)  # Addition
n2.left.right = Node(1.1092e+06, is_operator=False)  # Constant 1.1092e+06

# Left side of the addition
n2.left.left = Node('*', is_operator=True)  # Multiplication
n2.left.left.right = Node(92504, is_operator=False)  # Constant 92504

# Left side of the multiplication
n2.left.left.left = Node('*', is_operator=True)  # Another multiplication
n2.left.left.left.right = Node('x1', is_operator=False)  # x₁

# Left side of the multiplication
n2.left.left.left.left = Node('-', is_operator=True)  # Subtraction
n2.left.left.left.left.right = Node('x2', is_operator=False)  # x₂

# Left side of the subtraction
n2.left.left.left.left.left = Node('tan', is_operator=True)  # tan operator
n2.left.left.left.left.left.left = Node('sin', is_operator=True)  # sin operator
n2.left.left.left.left.left.left.left = Node('x0', is_operator=False)  # x₀


formula = Individual(n2, '../data/problem_2.npz')

print(formula)
mse_value = formula.mse('../data/problem_2.npz')
print("MSE value", mse_value)
print("MSE percentage", mse_percentage(mse_value, '../data/problem_2.npz'))

# return # Define the tree structure for the formula
n2 = Node('/', is_operator=True)

# Left side of division: multiplication
n2.left = Node('*', is_operator=True)

# Left side of multiplication: addition
n2.left.left = Node('+', is_operator=True)
n2.left.left.left = Node('x0', is_operator=False)
n2.left.left.right = Node(0.75024, is_operator=False)

# Add the missing term -0.12047 in the addition
n2.left.left.right = Node('+', is_operator=True)
n2.left.left.right.left = Node(0.75024, is_operator=False)
n2.left.left.right.right = Node(-0.12047, is_operator=False)

# Right side of multiplication: square
n2.left.right = Node('square', is_operator=True)

# Left side of square: subtraction
n2.left.right.left = Node('-', is_operator=True)
n2.left.right.left.left = Node(330.89, is_operator=False)

# Right side of subtraction: subtraction
n2.left.right.left.right = Node('-', is_operator=True)

# Left side of the second subtraction: division
n2.left.right.left.right.left = Node('/', is_operator=True)
n2.left.right.left.right.left.left = Node(-0.86566, is_operator=False)
n2.left.right.left.right.left.right = Node('x0', is_operator=False)

# Right side of the second subtraction: another subtraction
n2.left.right.left.right.right = Node('-', is_operator=True)
n2.left.right.left.right.right.left = Node('x0', is_operator=False)
n2.left.right.left.right.right.right = Node('square', is_operator=True)
n2.left.right.left.right.right.right.left = Node('x1', is_operator=False)

# Right side of square: multiplication
n2.left.right.left.right.right.right.right = Node('*', is_operator=True)
n2.left.right.left.right.right.right.right.left = Node('x1', is_operator=False)
n2.left.right.left.right.right.right.right.right = Node('x2', is_operator=False)

# Right side of division: constant 0.086551
n2.right = Node(0.086551, is_operator=False)

# Now `n2` holds the full tree structure

# Create the individual and evaluate it
formula = Individual(n2, '../data/problem_2.npz')
print(formula)
mse_value = formula.mse('../data/problem_2.npz')
print("MSE value", mse_value)
print("MSE percentage", mse_percentage(mse_value, '../data/problem_2.npz'))

#return (((((np.tan(np.sin(x[0])) - x[2]) * x[1]) * 92504) + 1109200.0) * x[0])


In [ ]:
# Define the tree structure for the formula
n2 = Node('/', is_operator=True)

# Left side of division: multiplication
n2.left = Node('*', is_operator=True)

# Left side of multiplication: addition
n2.left.left = Node('+', is_operator=True)
n2.left.left.left = Node('x0', is_operator=False)
n2.left.left.right = Node(0.75024, is_operator=False)

# Add the missing term -0.12047 in the addition
n2.left.left.right = Node('+', is_operator=True)
n2.left.left.right.left = Node(0.75024, is_operator=False)
n2.left.left.right.right = Node(-0.12047, is_operator=False)

# Right side of multiplication: square
n2.left.right = Node('square', is_operator=True)

# Left side of square: subtraction
n2.left.right.left = Node('-', is_operator=True)
n2.left.right.left.left = Node(330.89, is_operator=False)

# Right side of subtraction: subtraction
n2.left.right.left.right = Node('-', is_operator=True)

# Left side of the second subtraction: division
n2.left.right.left.right.left = Node('/', is_operator=True)
n2.left.right.left.right.left.left = Node(-0.86566, is_operator=False)
n2.left.right.left.right.left.right = Node('x0', is_operator=False)

# Right side of the second subtraction: another subtraction
n2.left.right.left.right.right = Node('-', is_operator=True)
n2.left.right.left.right.right.left = Node('x0', is_operator=False)
n2.left.right.left.right.right.right = Node('square', is_operator=True)
n2.left.right.left.right.right.right.left = Node('x1', is_operator=False)

# Right side of square: multiplication
n2.left.right.left.right.right.right.right = Node('*', is_operator=True)
n2.left.right.left.right.right.right.right.left = Node('x1', is_operator=False)
n2.left.right.left.right.right.right.right.right = Node('x2', is_operator=False)

# Right side of division: constant 0.086551
n2.right = Node(0.086551, is_operator=False)

# Now `n2` holds the full tree structure

# Create the individual and evaluate it
formula = Individual(n2, '../data/problem_2.npz')
print(formula)
mse_value = formula.mse('../data/problem_2.npz')
print("MSE value", mse_value)
print("MSE percentage", mse_percentage(mse_value, '../data/problem_2.npz'))

#return ((x[0] - 0.75024 + (-0.12047)) * np.square(330.89 - (-0.86566 / x[0]) - (x[0] - np.square(x[1])) - (x[1] * x[2]))) / 0.086551


In [ ]:
# Define the nodes for the formula
n3 = Node('+', is_operator=True)  # Outer addition

# First term: exp(sin((6.8302 - x₁)^2 + 3.9603))
n3.left = Node('exp', is_operator=True)  # Exponential
n3.left.left = Node('sin', is_operator=True)  # Sine function
n3.left.left.left = Node('+', is_operator=True)  # Addition inside sin
n3.left.left.left.right = Node(3.9603, is_operator=False)  # Constant 3.9603
n3.left.left.left.left = Node('square', is_operator=True)  # Square
n3.left.left.left.left.left = Node('-', is_operator=True)  # Subtraction (6.8302 - x₁)
n3.left.left.left.left.left.left = Node(6.8302, is_operator=False)  # Constant 6.8302
n3.left.left.left.left.left.right = Node('x1', is_operator=False)  # x₁

# Second term: (11.053 - x₂)
n3.right = Node('+', is_operator=True)  # Addition for remaining terms
n3.right.left = Node('-', is_operator=True)  # Subtraction (11.053 - x₂)
n3.right.left.left = Node(11.053, is_operator=False)  # Constant 11.053
n3.right.left.right = Node('x2', is_operator=False)  # x₂

# Third term: exp(0.00047697 - x₁) - exp(x₁) + x₀^2
n3.right.right = Node('+', is_operator=True)  # Addition inside third term
n3.right.right.right = Node('square', is_operator=True)  # x₀^2
n3.right.right.right.left = Node('x0', is_operator=False)  # x₀
n3.right.right.left = Node('-', is_operator=True)  # Subtraction of exponentials
n3.right.right.left.left = Node('exp', is_operator=True)  # exp(0.00047697 - x₁)
n3.right.right.left.left.left = Node('-', is_operator=True)  # Subtraction inside exp
n3.right.right.left.left.left.left = Node(0.00047697, is_operator=False)  # Constant 0.00047697
n3.right.right.left.left.left.right = Node('x1', is_operator=False)  # x₁
n3.right.right.left.right = Node('exp', is_operator=True)  # exp(x₁)
n3.right.right.left.right.left = Node('x1', is_operator=False)  # x₁


# Create the individual and evaluate it
formula = Individual(n3, '../data/problem_3.npz')
print(formula)
mse_value = formula.mse('../data/problem_3.npz')
print("MSE value", mse_value)
print("MSE percentage", mse_percentage(mse_value, '../data/problem_3.npz'))

#return np.exp(np.sin((6.8302 - x[1])**2 + 3.9603)) + (11.053 - x[2]) + ((np.exp(0.000477 - x[1]) - np.exp(x[1])) + x[0]**2)



In [ ]:
# Define the nodes for the formula
n4 = Node('-', is_operator=True)  # Outer subtraction
n4.right = Node(-2.2033, is_operator=False)  # Constant -2.2033

# Left side of the outer subtraction
n4.left = Node('+', is_operator=True)  # Outer addition
n4.left.right = Node(1.0745, is_operator=False)  # Constant 1.0745

# Left side of the addition
n4.left.left = Node('/', is_operator=True)  # Division
n4.left.left.right = Node(1.4176, is_operator=False)  # Constant 1.4176

# Left side of the division
n4.left.left.left = Node('/', is_operator=True)  # Another division
n4.left.left.left.right = Node(0.2017, is_operator=False)  # Constant 0.2017

# Left side of the second division
n4.left.left.left.left = Node('+', is_operator=True)  # Addition
n4.left.left.left.left.left = Node('cos', is_operator=True)  # Cosine of (x₁ + -0.0018035)
n4.left.left.left.left.left.left = Node('+', is_operator=True)  # Addition inside cos
n4.left.left.left.left.left.left.left = Node('x1', is_operator=False)  # x₁
n4.left.left.left.left.left.left.right = Node(-0.0018035, is_operator=False)  # Constant -0.0018035
n4.left.left.left.left.right = Node('cos', is_operator=True)  # Cosine of x₁
n4.left.left.left.left.right.left = Node('x1', is_operator=False)  # x₁



# Create the individual and evaluate it
formula = Individual(n4, '../data/problem_4.npz')
print(formula)
mse_value = formula.mse('../data/problem_4.npz')
print("MSE value", mse_value)
print("MSE percentage", mse_percentage(mse_value, '../data/problem_4.npz'))

# return (((np.cos(x[1] + -0.0018035) + np.cos(x[1])) / 0.2017) / 1.4176) + 1.0745 - (-2.2033)


In [ ]:
# Define the nodes for the formula
n4 = Node('+', is_operator=True)  # Outer addition
n4.right = Node(3.2925, is_operator=False)  # Constant 3.2925

# Left side of the outer addition
n4.left = Node('*', is_operator=True)  # Multiplication
n4.left.right = Node(-0.030303, is_operator=False)  # Constant -0.030303

# Left side of the multiplication
n4.left.left = Node('+', is_operator=True)  # Addition
n4.left.left.right = Node('-', is_operator=True)  # Subtraction: ((x₀ - x₀) - x₀)
n4.left.left.right.left = Node('-', is_operator=True)  # First subtraction: (x₀ - x₀)
n4.left.left.right.left.left = Node('x0', is_operator=False)  # x₀
n4.left.left.right.left.right = Node('x0', is_operator=False)  # x₀
n4.left.left.right.right = Node('x0', is_operator=False)  # x₀
n4.left.left.right.left.right = Node(-0.01839, is_operator=False)  # Constant -0.01839

# Left side of the addition
n4.left.left.left = Node('-', is_operator=True)  # Subtraction: exp(cos(x₁)) - x₀
n4.left.left.left.left = Node('exp', is_operator=True)  # Exponential
n4.left.left.left.left.left = Node('cos', is_operator=True)  # Cosine
n4.left.left.left.left.left.left = Node('x1', is_operator=False)  # x₁
n4.left.left.left.right = Node('x0', is_operator=False)  # x₀



# Create the individual and evaluate it
formula = Individual(n4, '../data/problem_4.npz')
print(formula)
mse_value = formula.mse('../data/problem_4.npz')
print("MSE value", mse_value)
print("MSE percentage", mse_percentage(mse_value, '../data/problem_4.npz'))


#return (np.exp(np.cos(x[1])) - x[0] + (((x[0] - x[0]) - x[0]) - -0.01839)) * -0.030303 + 3.2925

In [ ]:
# Define the nodes for the formula
n5 = Node('sin', is_operator=True)  # Outer sin
n5.left = Node('sin', is_operator=True)  # Inner sin

# Left side of the inner sin
n5.left.left = Node('*', is_operator=True)  # Multiplication
n5.left.left.right = Node(-6.4885e-13, is_operator=False)  # Constant -6.4885e-13

# Left side of the multiplication
n5.left.left.left = Node('exp', is_operator=True)  # Exponential
n5.left.left.left.left = Node('+', is_operator=True)  # Addition
n5.left.left.left.left.left = Node('x1', is_operator=False)  # x₁
n5.left.left.left.left.right = Node('x0', is_operator=False)  # x₀

# Create the individual and evaluate it
formula = Individual(n5, '../data/problem_5.npz')
print(formula)
mse_value = formula.mse('../data/problem_5.npz')
print("MSE value", mse_value)
print("MSE percentage", mse_percentage(mse_value, '../data/problem_5.npz'))



In [ ]:
# Define the root node for the formula
n6 = Node('+', is_operator=True)  # Outer addition

# Left side of the addition
n6.left = Node('+', is_operator=True)  # (x₁ + x₁)
n6.left.left = Node('x1', is_operator=False)  # x₁
n6.left.right = Node('x1', is_operator=False)  # x₁

# Right side of the addition
n6.right = Node('-', is_operator=True)  # 0.036213 - (...)
n6.right.left = Node(0.036213, is_operator=False)  # Constant 0.036213
n6.right.right = Node('+', is_operator=True)  # Inner addition (...) + x₀
n6.right.right.left = Node('*', is_operator=True)  # Multiplication (...)
n6.right.right.left.left = Node('-', is_operator=True)  # Subtraction (...)
n6.right.right.left.left.left = Node('-', is_operator=True)  # (x₀ - 0.11854)
n6.right.right.left.left.left.left = Node('x0', is_operator=False)  # x₀
n6.right.right.left.left.left.right = Node(0.11854, is_operator=False)  # Constant 0.11854
n6.right.right.left.left.right = Node('-', is_operator=True)  # (x₁ - 9.445e-08)
n6.right.right.left.left.right.left = Node('x1', is_operator=False)  # x₁
n6.right.right.left.left.right.right = Node(9.445e-08, is_operator=False)  # Constant 9.445e-08
n6.right.right.left.right = Node(-0.30548, is_operator=False)  # Constant -0.30548
n6.right.right.right = Node('x0', is_operator=False)  # x₀

# Create the individual and evaluate it
formula = Individual(n6, '../data/problem_6.npz')
print("Tree structure created successfully.")
print("Formula:", formula)
mse_value = formula.mse('../data/problem_6.npz')
print("MSE value:", mse_value)
print("MSE percentage:", mse_percentage(mse_value, '../data/problem_6.npz'))


In [ ]:
# Outer addition
n7 = Node('+', is_operator=True)

# Left side: cos(x₀ * ((x₀ / x₁) * -0.59086))
n7.left = Node('cos', is_operator=True)
n7.left.left = Node('*', is_operator=True)
n7.left.left.left = Node('x0', is_operator=False)  # x₀
n7.left.left.right = Node('*', is_operator=True)
n7.left.left.right.left = Node('/', is_operator=True)
n7.left.left.right.left.left = Node('x0', is_operator=False)  # x₀
n7.left.left.right.left.right = Node('x1', is_operator=False)  # x₁
n7.left.left.right.right = Node(-0.59086, is_operator=False)  # Constant -0.59086

# Right side: exp(((x₀ * x₁) - -0.82405) * 1.1823) + 1.0143
n7.right = Node('+', is_operator=True)
n7.right.left = Node('exp', is_operator=True)
n7.right.left.left = Node('*', is_operator=True)
n7.right.left.left.left = Node('-', is_operator=True)
n7.right.left.left.left.left = Node('*', is_operator=True)
n7.right.left.left.left.left.left = Node('x0', is_operator=False)  # x₀
n7.right.left.left.left.left.right = Node('x1', is_operator=False)  # x₁
n7.right.left.left.left.right = Node(-0.82405, is_operator=False)  # Constant -0.82405
n7.right.left.left.right = Node(1.1823, is_operator=False)  # Constant 1.1823
n7.right.right = Node(1.0143, is_operator=False)  # Constant 1.0143


# Create the individual and evaluate it
formula = Individual(n7, '../data/problem_7.npz')
print("Tree structure created successfully.")
print("Formula:", formula)
mse_value = formula.mse('../data/problem_7.npz')
print("MSE value:", mse_value)
print("MSE percentage:", mse_percentage(mse_value, '../data/problem_7.npz'))


In [ ]:
# Outer addition
n7 = Node('+', is_operator=True)

# Left side: exp(((cos(x₁ * (x₁ - 0.13746)) + cos(x₀ * x₀)) * -0.941) + (((x₁ * 0.78139) * x₀) - -0.65089))
n7.left = Node('exp', is_operator=True)
n7.left.left = Node('+', is_operator=True)

# First term: ((cos(x₁ * (x₁ - 0.13746)) + cos(x₀ * x₀)) * -0.941)
n7.left.left.left = Node('*', is_operator=True)
n7.left.left.left.left = Node('+', is_operator=True)
n7.left.left.left.left.left = Node('cos', is_operator=True)
n7.left.left.left.left.left.left = Node('*', is_operator=True)
n7.left.left.left.left.left.left.left = Node('x1', is_operator=False)  # x₁
n7.left.left.left.left.left.left.right = Node('-', is_operator=True)
n7.left.left.left.left.left.left.right.left = Node('x1', is_operator=False)  # x₁
n7.left.left.left.left.left.left.right.right = Node(0.13746, is_operator=False)  # Constant 0.13746
n7.left.left.left.left.right = Node('cos', is_operator=True)
n7.left.left.left.left.right.left = Node('*', is_operator=True)
n7.left.left.left.left.right.left.left = Node('x0', is_operator=False)  # x₀
n7.left.left.left.left.right.left.right = Node('x0', is_operator=False)  # x₀
n7.left.left.left.right = Node(-0.941, is_operator=False)  # Constant -0.941

# Second term: (((x₁ * 0.78139) * x₀) - -0.65089)
n7.left.left.right = Node('-', is_operator=True)
n7.left.left.right.left = Node('*', is_operator=True)
n7.left.left.right.left.left = Node('*', is_operator=True)
n7.left.left.right.left.left.left = Node('x1', is_operator=False)  # x₁
n7.left.left.right.left.left.right = Node(0.78139, is_operator=False)  # Constant 0.78139
n7.left.left.right.left.right = Node('x0', is_operator=False)  # x₀
n7.left.left.right.right = Node(-0.65089, is_operator=False)  # Constant -0.65089

# Right side: Constant 4.0888
n7.right = Node(4.0888, is_operator=False)



# Create the individual and evaluate it
formula = Individual(n7, '../data/problem_7.npz')
print("Tree structure created successfully.")
print("Formula:", formula)
mse_value = formula.mse('../data/problem_7.npz')
print("MSE value:", mse_value)
print("MSE percentage:", mse_percentage(mse_value, '../data/problem_7.npz'))


In [ ]:
# Outer subtraction
n8 = Node('-', is_operator=True)

# Left side: (square(square(x₅ * 1.4953)) * x₅)
n8.left = Node('*', is_operator=True)
n8.left.left = Node('square', is_operator=True)
n8.left.left.left = Node('square', is_operator=True)
n8.left.left.left.left = Node('*', is_operator=True)
n8.left.left.left.left.left = Node('x5', is_operator=False)  # x₅
n8.left.left.left.left.right = Node(1.4953, is_operator=False)  # Constant 1.4953
n8.left.right = Node('x5', is_operator=False)  # x₅

# Right side: (square((x₄ * -2.006) * x₄) + ((x₁ * x₁) + (x₃ * -45.162)))
n8.right = Node('+', is_operator=True)

# First term: square((x₄ * -2.006) * x₄)
n8.right.left = Node('square', is_operator=True)
n8.right.left.left = Node('*', is_operator=True)
n8.right.left.left.left = Node('*', is_operator=True)
n8.right.left.left.left.left = Node('x4', is_operator=False)  # x₄
n8.right.left.left.left.right = Node(-2.006, is_operator=False)  # Constant -2.006
n8.right.left.left.right = Node('x4', is_operator=False)  # x₄

# Second term: ((x₁ * x₁) + (x₃ * -45.162))
n8.right.right = Node('+', is_operator=True)
n8.right.right.left = Node('*', is_operator=True)
n8.right.right.left.left = Node('x1', is_operator=False)  # x₁
n8.right.right.left.right = Node('x1', is_operator=False)  # x₁
n8.right.right.right = Node('*', is_operator=True)
n8.right.right.right.left = Node('x3', is_operator=False)  # x₃
n8.right.right.right.right = Node(-45.162, is_operator=False)  # Constant -45.162



# Create the individual and evaluate it
formula = Individual(n8, '../data/problem_8.npz')
print("Tree structure created successfully.")
print("Formula:", formula)
mse_value = formula.mse('../data/problem_8.npz')
print("MSE value:", mse_value)
print("MSE percentage:", mse_percentage(mse_value, '../data/problem_8.npz'))